In [ ]:
!pip install transformers torch nltk rouge_score

In [ ]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import fastText
from contextlib import contextmanager
import time
import json
# import wandb
import torch.nn.functional as F
import torch
from collections import deque
import numpy as np

In [ ]:
from power_monitoring.monitor import HWMonitor
import threading
# import wandb

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
# combined_outputs["gsm8k"] = [{"input_text": None, "7b": None, "tiny": None, "13b": None} for _ in range(3000)]
# len(combined_outputs["gsm8k"])

# with open('data/input_output_train', 'wb') as f:
#     pickle.dump(combined_outputs, f)

In [ ]:
import pickle
import pandas as pd

with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

In [ ]:
df_combined_outputs.size

In [ ]:
df_combined_outputs["cnn_dailymail"][3]

In [ ]:
def algorithm(T, V, h_tilde, c):
    l_predictor, s_predictor = None, None
    Q = 0.0 
    results = []

    # l_loss_window = []
    # s_loss_window = []
    acc_all = []
    energy_all = []

    for t in range(1, T+1):
        print(f"Index: {t}")
        t_i = df_combined_outputs["wmt14"][t]["input_text"]
        t_e = df_combined_outputs["wmt14"][t]["label"]

        p_t = min(1, c / math.sqrt(t))
        X_t = Bernoulli(p_t)

        # l_loss, s_loss = None, None

        g_large = 2.0
        g_small = 1.0  

        # if X_t == 1:
        #     t_c = getResults(t, t_i, t_e)
        #     l_actual_acc = t_c['large_model_accuracy']
        #     s_actual_acc = t_c['small_model_accuracy']

        #     l_predictor, s_predictor, l_loss, s_loss = sgdStep(l_predictor, s_predictor, t_c)
        #     l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)

        #     cost_large = V * g_large + Q * (h_tilde - l_actual_acc)
        #     cost_small = V * g_small + Q * (h_tilde - s_actual_acc)
        # else:
        # l_predicted_acc, s_predicted_acc = predict(t_i, l_predictor, s_predictor)

        t_c = getResults(t, t_i, t_e)
        l_actual_acc = t_c['large_model_accuracy']
        s_actual_acc = t_c['small_model_accuracy']
        cost_large = V * g_large + Q * (h_tilde - l_actual_acc)
        cost_small = V * g_small + Q * (h_tilde - s_actual_acc)

        print('Q=', Q)
        if cost_large < cost_small:
            x_t = 'large'
            selected_acc = l_actual_acc # if X_t == 1 else l_predicted_acc
            selected_energy = g_large
            print('selected large')
        else:
            x_t = 'small'
            selected_acc = s_actual_acc # if X_t == 1 else s_predicted_acc
            selected_energy = g_small
            print('selected small')

        output, t_acc = querySelectedModel(t, t_e, x_t)
        # t_acc = selected_acc
        # output = ''

        # if l_loss is not None:
        #     l_loss_window.append(l_loss)
        # if s_loss is not None:
        #     s_loss_window.append(s_loss)
        acc_all.append(t_acc)
        energy_all.append(selected_energy)

        # smoothed_l_loss = moving_average(l_loss_window, window_size=1000)
        # smoothed_s_loss = moving_average(s_loss_window, window_size=1000)
        avg_acc = np.mean(acc_all)
        avg_energy = np.mean(energy_all)

        Q = max(0.0, Q + h_tilde - t_acc)
        log(results, t, t_i, output, avg_acc, avg_energy, x_t) #, smoothed_l_loss, smoothed_s_loss)

    return results

In [ ]:
def log(results, t, t_i, output, t_acc, t_energy, x_t): #, l_loss, s_loss):
    results.append({
        'iteration': t,
        'input': t_i,
        'best_output': output,
        'chosen_models_avg_accuracy': t_acc,
        'chosen_models_avg_energy': t_energy,
        'chosen_model': x_t,
        # 'l_loss': l_loss,
        # 's_loss': s_loss
    })
    # log_data = {
    #     'iteration': t,
    #     'chosen_model_accuracy': t_acc
    # }
        
    # if l_loss is not None: log_data['l_loss'] = l_loss
    # if s_loss is not None: log_data['s_loss'] = s_loss
        
    # wandb.log(log_data)

In [ ]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [ ]:
def querySelectedModel(t, t_e, x_t):
    if x_t == "large":
        output = df_combined_outputs["wmt14"][t]["13b"]
    else:
        output =  df_combined_outputs["wmt14"][t]["tiny"]
        
    return output, calculate_bleu(output, t_e)

In [ ]:
# def queryBest(t, t_i, t_e, l_predictor, s_predictor):
#     l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
#     if l_acc > s_acc: # use larger model, e.g. 7b llama
#         output = df_combined_outputs["wmt14"][t]["13b"]
#     else: # use smaller model for all other cases, e.g. 3b llama
#         output =  df_combined_outputs["wmt14"][t]["tiny"]

#     return output, calculate_bleu(output, t_e)
#     # return output, calculate_rouge(output, t_e)

In [ ]:
def moving_average(data, window_size):
    data_list = list(data)  
    if len(data_list) == 0:
        return 0.0 
    if len(data_list) < window_size:
        return sum(data_list) / len(data_list)
    return sum(data_list[-window_size:]) / window_size

In [ ]:
def compute_mse_loss(predicted_accuracy, true_accuracy):
    predicted_tensor = torch.tensor([predicted_accuracy], dtype=torch.float32)
    true_tensor = torch.tensor([true_accuracy], dtype=torch.float32)

    loss = F.mse_loss(predicted_tensor, true_tensor)
    
    return loss.item()

In [ ]:
def sgdStep(large_model_predictor, small_model_predictor, t_c):
    predicted_large_label, predicted_small_label = 0, 0
    if large_model_predictor and small_model_predictor != None: 
        predicted_large_label = large_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
        predicted_small_label = small_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
    
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    large_model_loss = compute_mse_loss(float(predicted_large_label), t_c['large_model_accuracy'])
    small_model_loss = compute_mse_loss(float(predicted_small_label), t_c['small_model_accuracy'])

    # print(f"Large Stats(predicted, actual): {predicted_large_label}, {t_c['large_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")
    # print(f"Small Stats(predicted, actual): {predicted_small_label}, {t_c['small_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")

    return large_model_predictor, small_model_predictor, large_model_loss, small_model_loss

In [ ]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [ ]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["wmt14"][t]["tiny"]
    l_output = df_combined_outputs["wmt14"][t]["13b"]
    
    # WMT14
    s_acc = calculate_bleu(s_output, t_e)
    l_acc = calculate_bleu(l_output, t_e)

    # CNN Dailymail
    # s_acc = calculate_rouge(s_output, t_e)['rouge1']
    # l_acc = calculate_rouge(l_output, t_e)['rouge1']

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [ ]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [ ]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [ ]:
def run_and_record(T, V, h_tilde, c):
    # wandb.init(project="classifier")
    # stop_event = threading.Event()
    # hw_monitor = HWMonitor(monitoring_freq=1.0, stop_event=threading.Event())
    # hw_monitor.start()

    
    results = algorithm(T, V, h_tilde, c)

    # stop_event.set() 
    # hw_monitor.join()
    # wandb.finish()

    return results

In [ ]:
h_tilde_config = 0.2

In [ ]:
results = run_and_record(2999, 0.01, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V0.01' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 0.1, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V0.1' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 1, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V1' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 10, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V10' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 100, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V100' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 1000, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V1000' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 10000, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V10000' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 100000, h_tilde_config, 5)
with open('experiments/fasttext/wmt14_results_V100000' + '_h' + str(h_tilde_config) + '.json', 'w') as f:
    json.dump(results, f, indent=4)